<span style="font-size:large;">
<div style="text-align: right">IM-UH 1511 <b>Introduction to Digital Humanities</b><br/>
Student name: <span style="color:blue"><b>X XX</b></span><br/>
Grade: <span style="color:red"><b></b></span><br/><br/></div>

## <span style="color:red">HOMEWORK 1</span>
# <span style="color:green">Extraction of Names from Text

In [1]:
import time
start_time = time.perf_counter()
import urllib, os, codecs, random, operator, re, string, copy, dateutil.parser, itertools, pickle, datetime, math, pandas as pd, numpy as np, matplotlib.pyplot as plt
from collections import Counter
from string import punctuation, digits
import pathlib
import spacy
from spacy import displacy
nlp = spacy.load('en_core_web_lg') # !python3 -m spacy download en_core_web_lg
import inflect
import nltk
from nltk import word_tokenize

import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning) 
warnings.simplefilter('ignore')

### Load Data

In [2]:
# get your working directory
home = str(pathlib.Path.cwd())

# create a path to which the file will be written
text_path = os.path.join(home, 'Dracula.txt')

# location of the project gutenberg copy of the moby-dick text file
text_url = 'http://www.gutenberg.org/cache/epub/345/pg345.txt'

urllib.request.urlretrieve(text_url, text_path)

print('Downloaded to:', text_path)

Downloaded to: /Users/moses/WorkPlaces/Python Projects 2/3 NYUAD Digital Humanities/Homework1 NamesExtraction/Dracula.txt


In [3]:
f = codecs.open(text_path, "r", encoding="utf-8").readlines()
for line in f:
    if line.startswith("(_Kept in shorthand._)"):
        print(f.index(line)) #198
    if line.startswith("                                THE END"):
        print(f.index(line)) #15514

198
15514


In [4]:
ff=f[194:15514]
ff

['CHAPTER I\r\n',
 '\r\n',
 "JONATHAN HARKER'S JOURNAL\r\n",
 '\r\n',
 '(_Kept in shorthand._)\r\n',
 '\r\n',
 '\r\n',
 '_3 May. Bistritz._--Left Munich at 8:35 P. M., on 1st May, arriving at\r\n',
 'Vienna early next morning; should have arrived at 6:46, but train was an\r\n',
 'hour late. Buda-Pesth seems a wonderful place, from the glimpse which I\r\n',
 'got of it from the train and the little I could walk through the\r\n',
 'streets. I feared to go very far from the station, as we had arrived\r\n',
 'late and would start as near the correct time as possible. The\r\n',
 'impression I had was that we were leaving the West and entering the\r\n',
 'East; the most western of splendid bridges over the Danube, which is\r\n',
 'here of noble width and depth, took us among the traditions of Turkish\r\n',
 'rule.\r\n',
 '\r\n',
 'We left in pretty good time, and came after nightfall to Klausenburgh.\r\n',
 'Here I stopped for the night at the Hotel Royale. I had for dinner, or\r\n',
 'rathe

In [5]:
text="\r\n".join(ff)
text

'CHAPTER I\r\n\r\n\r\n\r\nJONATHAN HARKER\'S JOURNAL\r\n\r\n\r\n\r\n(_Kept in shorthand._)\r\n\r\n\r\n\r\n\r\n\r\n_3 May. Bistritz._--Left Munich at 8:35 P. M., on 1st May, arriving at\r\n\r\nVienna early next morning; should have arrived at 6:46, but train was an\r\n\r\nhour late. Buda-Pesth seems a wonderful place, from the glimpse which I\r\n\r\ngot of it from the train and the little I could walk through the\r\n\r\nstreets. I feared to go very far from the station, as we had arrived\r\n\r\nlate and would start as near the correct time as possible. The\r\n\r\nimpression I had was that we were leaving the West and entering the\r\n\r\nEast; the most western of splendid bridges over the Danube, which is\r\n\r\nhere of noble width and depth, took us among the traditions of Turkish\r\n\r\nrule.\r\n\r\n\r\n\r\nWe left in pretty good time, and came after nightfall to Klausenburgh.\r\n\r\nHere I stopped for the night at the Hotel Royale. I had for dinner, or\r\n\r\nrather supper, a chicken 

In [6]:
titlename = "Bram Stoker's Dracula"

words = word_tokenize(text)
nuw=len(words)
uw=len(set(words))
print("%s contains %i nonunique and %i unique words"%(titlename,nuw,uw))

Bram Stoker's Dracula contains 189685 nonunique and 10627 unique words


### Extraction of Proper Nouns

In [ ]:
p = inflect.engine()
d_tags = {}

docs_d={"Dracula":text}
for key, value in docs_d.items():
    arr = []
    doc = nlp(value.replace('\n',''))
    #Keep these types of nlp entities
    keep_l = ['PERSON'] #,'NORP','PRODUCT','ORG']
    #Typo/model error + german corrections
    drop_t = []
    
    #Things inflect library handles poorly or to exclude from touching
    ex_ls = []
    
    for X in doc.ents:
        s1 = X.text
        if (X.label_ in keep_l) and (s1.lower() not in drop_t) and (s1):
            arr.append((s1, X.label_))
    d_tags[key] = arr
# pprint(d_tags)
names=[]
for k,v in d_tags.items():
    for vv in v:
        if vv[0] not in names:
            p=vv[0].replace("'","")
            p=p.title()
            names.append(p)
names=sorted(set(names))
print(len(names))
names

In [ ]:
rem=[]
for p in names:
    if "_" in p:
        rem.append(p)
    if "--" in p:
        rem.append(p)
    if p not in text:
        rem.append(p)
names=[p for p in names if p not in rem]
pp=[q for q in itertools.product(names,names) if q[0]!=q[1]]
for q in pp:
    if q[0] in q[1]:
        rem.append(q[0])
    if q[1] in q[0]:
        rem.append(q[1])
    w=q[0]+" "+q[1]
    if w in text:
        names.append(w)
        rem.append(q[0])
        rem.append(q[1])
names=[p for p in names if p not in rem]
names=sorted(set(names))
print(len(names))
names

In [ ]:
rem=['Breakfast','Chin','Crucifix','Wafer','Ye','D. Lit','Devil','Draculas',
    'Friend Arthur','H. B. M.','Harker Jonathan','Herr','Harkers','Hun',
    'I. He','Ittin','Lively','Lookin','Lordship','M. D.','M. R. C. S. L. K. Q. C. P. I.',
    'Manlike','Mein Gott','Mem','Moneybag','Moon','Nature','Morris Quincey',
    'Nay','Omne','Ounds','Pass','Seein','Telegram','Yus','Arthur Ho','Cszeks',
    'Stay','Stop','Soh','This Braithwaite Lowrey','Lord God','Robin Ho',
    'Quincey P. Morris','Faugh','Mitchell','Jonathan Harker','Mina Harker',
    'Sister','Agatha',"Joseph","Miss Westenra"]
names=[p for p in names if p not in rem]
names=names+['Robin Hood','Soho','Braithwaite Lowrey','Van Helsing',
            'Mitchell, Sons, & Candy','Jonathan','Mina','Sister Agatha',
            'Count Dracula','Ste. Mary','St. Joseph',"Saxon"]
names=sorted(set(names))
print(len(names))
names

In [ ]:
nfreq=[]
for i in names: 
    nfreq.append(text.count(i))
pnf_df = pd.DataFrame(
    {'Names': names, 
     'Frequency of Occurrences': nfreq
    })
pnf_df=pnf_df[['Names','Frequency of Occurrences']]
pnf_df=pnf_df.sort_values(by ='Frequency of Occurrences',ascending=False)
# trf_df=trf_df[trf_df["Frequency of Occurrences"]>10]
print(len(pnf_df))
pnf_df[:50]

In [ ]:
pnf_df.to_csv('Names_freqs.csv')

In [ ]:
print("Run in %.2f seconds (%.2f minutes)" %(time.perf_counter() - start_time,(time.perf_counter() - start_time)/60))